In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [2]:
timer_all = Timer()
timer_all.reset()

In [3]:
timer.reset()
tb = TbPacientes()
# tb.update()
timer.stop()

133.48810982704163

In [4]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

Casos confirmados excluidos: 1
Obitos confirmados excluidos: 2


226.73935651779175

In [5]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (474056, 21)
casos tb_pacientes: (471640, 48)


In [6]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [7]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [8]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)

casos_confirmados_nao_tb_pacientes: (12, 21)
tb_pacientes_nao_casos_confirmados: (1, 48)


In [9]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [10]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [11]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
# exit()

TypeError: 'bool' object is not iterable

In [ ]:
timer.reset()

sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
for mun, df_mun in casosc.groupby('ibge7'):
    sorted_casosc = sorted_casosc.append(df_mun.sort_values('nome_hash'))

sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
for mun, df_mun in casost.groupby('ibge_res_pr'):
    sorted_casost = sorted_casost.append(df_mun.sort_values('nome_hash'))

casosc = sorted_casosc
print(f"casos casos_confirmados: {sorted_casosc.shape}")
casost = sorted_casost
print(f"casos tb_pacientes: {casost.shape}")
print(timer.stop())

In [ ]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))

In [ ]:
casosc = casosc.drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])
casost = casost.drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])

In [ ]:
casosc['idade_mais1'] = casosc['idade'] + 1
casosc['idade_menos1'] = casosc['idade'] - 1

In [ ]:
writer = pd.ExcelWriter(join(output,"novos_tb_pacientes.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casost.to_excel(writer,index=None)

writer.save()
writer.close()

In [ ]:
writer = pd.ExcelWriter(join(output,"casos_ordenados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc.to_excel(writer,index=None)

writer.save()
writer.close()

In [ ]:
print(timedelta(seconds=timer_all.stop()))
exit()

In [ ]:
# casost = tb.get()
# casost['nome_hash'] = casost['nome'].apply(normalize_hash)
writer = pd.ExcelWriter("obitos_tb.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


obitos_tb = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)

for dt, df in casost.loc[casost['obito']=='SIM'].sort_values('dt_obito').groupby('dt_obito'):
    obitos_tb = obitos_tb.append(df.sort_values('nome_hash'))

obitos_tb[['nome','sexo','idade','mun_resid','telefone','dt_obito']].to_excel(writer,index=None)
writer.save()
writer.close()

In [ ]:
obitos_tb